<b>STEP 1:</b> Connecting to the AWS data catalog via STAC to search for the datasets satisfying the following criteria:
<ul>
  <li style="font-size:18px">Period between 2021-10-08 and 2022-05-05.</li>
  <li style="font-size:18px">Covering more that 90% of the study area.</li>
  <li style="font-size:18px">With no clouds.</li>
</ul>

In [244]:
from pystac_client import Client
bbox = [-8.4197, 32.6703, -8.4122, 32.6656]
time = ["2021-10-08", "2022-05-05"]
catalog = Client.open("https://earth-search.aws.element84.com/v0")
query = catalog.search(
    collections=["sentinel-s2-l2a-cogs"], #-cogs
    datetime=time,
    limit=100,
    bbox=bbox,
    query = {"sentinel:data_coverage":{"gt":"90"}})
items = list(query.get_items())
print(f"Found: {len(items):d} items")

Found: 42 items


<center><b>--------------------------------------------------------------------</b></center>
<br>
<b>STEP 2:</b> Loading the data from the query used in STAC earlier and creating a subset of the datasets with a resolution of 10m to keep only an experimental plot with an area of 20 hectare situated in the Sidi Bennour District:

In [245]:
from odc.stac import configure_rio, stac_load
data = stac_load(
    items,
    crs="epsg:32629",
    bands=("B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "SCL"),
    resolution=10,
    chunks={},
    groupby="solar_day",
    bbox=bbox,)
#data

<xarray.Dataset>
Dimensions:      (y: 53, x: 71, time: 42)
Coordinates:
  * y            (y) float64 3.615e+06 3.615e+06 ... 3.614e+06 3.614e+06
  * x            (x) float64 5.544e+05 5.544e+05 ... 5.551e+05 5.551e+05
    spatial_ref  int32 32629
  * time         (time) datetime64[ns] 2021-10-08T11:22:19 ... 2022-05-01T11:...
Data variables:
    B01          (time, y, x) float32 dask.array<chunksize=(1, 53, 71), meta=np.ndarray>
    B02          (time, y, x) float32 dask.array<chunksize=(1, 53, 71), meta=np.ndarray>
    B03          (time, y, x) float32 dask.array<chunksize=(1, 53, 71), meta=np.ndarray>
    B04          (time, y, x) float32 dask.array<chunksize=(1, 53, 71), meta=np.ndarray>
    B05          (time, y, x) float32 dask.array<chunksize=(1, 53, 71), meta=np.ndarray>
    B06          (time, y, x) float32 dask.array<chunksize=(1, 53, 71), meta=np.ndarray>
    B07          (time, y, x) float32 dask.array<chunksize=(1, 53, 71), meta=np.ndarray>
    B08          (time, y, x) float32 dask.array<chunksize=(1, 53, 71), meta=np.ndarray>
    SCL          (time, y, x) float32 dask.array<chunksize=(1, 53, 71), meta=np.ndarray>

In [246]:
import matplotlib.pyplot as plt

#aa=data.isel(time=0).SCL
#ab=np.array(aa)
#plt.imshow(ab)
#plt.colorbar()
#plt.clim(0, 11)
#plt.show()

<center><b>--------------------------------------------------------------------</b></center>
<br>
<b>STEP 3:</b> Calculate NDVI for all products and save the outputs on a local folder:

In [247]:
import numpy as np
import rasterio

array_NDVI = data.isel(time=0).B02
# Gather array dimensions
height, width = array_NDVI.shape

In [249]:
import matplotlib.pyplot as plt

for i in range(data.coords["time"].size):
    scl = data.isel(time=i).SCL
    npscl = np.array(scl)
    
    arr = np.array(data.coords["time"][i])
    
    #plt.imshow(npscl)
    #plt.colorbar()
    #plt.clim(0, 11)
    #plt.title(str(arr)[:-19])
    #plt.show()

In [250]:
# Metadata
metadata = {
    'driver': 'GTiff',
    'dtype': rasterio.float32,
    'count': 1,
    'height': height-1,
    'width': width-1,
    'transform': rasterio.Affine(10, 0, 554415, 0, -10, 3614888),  # Transformation par défaut pour une grille régulière
    'crs': rasterio.crs.CRS.from_epsg(32629)  # Système de coordonnées de référence (par exemple, EPSG:4326 pour WGS84)
}
# NDVI computation and data export
for i in range(data.coords["time"].size):
    arr = str(np.array(data.coords["time"][i]))[:-19]
    
    array_B01 = data.isel(time=i).B01
    array_B02 = data.isel(time=i).B02
    array_B03 = data.isel(time=i).B03
    array_B04 = data.isel(time=i).B04
    array_B05 = data.isel(time=i).B05
    array_B06 = data.isel(time=i).B06
    array_B07 = data.isel(time=i).B07
    array_B08 = data.isel(time=i).B08
    array_SCL = data.isel(time=i).SCL
    
    # cloud control
    scl = np.array(array_SCL)
    nancount = np.count_nonzero(scl >= 8)
    if nancount == 0:
    
        # derived indices
        array_NDVI= (array_B08 - array_B04)/(array_B08 + array_B04)
        array_alb = 0.130*array_B01 + 0.115*array_B02 + 0.143*array_B03 + 0.180*array_B04 + 0.281*array_B05 + 0.108*array_B06 + 0.042*array_B07
        array_alb = array_alb/10000
        NDVI_s=0.025 # -0.15
        NDVI_v=1.1 # 0.90
        array_fv = (array_NDVI-NDVI_s) / (NDVI_v-NDVI_s)
        array_lai = -np.log(1-array_fv) / 0.5
    
        # NDVI (reshape and write)
        reshaper = np.array(array_NDVI)
        #reshaper [scl>=8] = None
        reshaper = np.delete(reshaper,0,0) # delete last row
        reshaper = np.delete(reshaper,0,1) # delete first column
        array_NDVI = reshaper
        # Writing Output NDVI
        output_file = "/home/eoafrica/shared/Final Workflow/NDVI/Sidi_Bennour_NDVI_{0}.tif".format(arr)
        #print("NDVI: ",arr,"........100%")
        # Creation of TIFF file
        with rasterio.open(output_file, 'w', **metadata) as dst:
            dst.write(array_NDVI, 1)  # Writing in band 1
        
        # Albedo (reshape and write)
        reshaper = np.array(array_alb)
        #reshaper [scl>=8] = None
        reshaper = np.delete(reshaper,0,0) # delete last row
        reshaper = np.delete(reshaper,0,1) # delete first column
        array_alb = reshaper
        # Writing Output albedo
        output_file = "/home/eoafrica/shared/Final Workflow/ALBEDO/Sidi_Bennour_ALB_{0}.tif".format(arr)
        #print("Albedo: ",arr,"........100%")
        # Creation of TIFF file
        with rasterio.open(output_file, 'w', **metadata) as dst:
            dst.write(array_alb, 1)  # Writing in band 1
        
        # FV (reshape and write)
        reshaper = np.array(array_fv)
        #reshaper [scl>=8] = None
        reshaper = np.delete(reshaper,0,0) # delete last row
        reshaper = np.delete(reshaper,0,1) # delete first column
        array_fv = reshaper
        # Writing Output albedo
        output_file = "/home/eoafrica/shared/Final Workflow/FV/Sidi_Bennour_FV_{0}.tif".format(arr)
        #print("FVC: ",arr,"........100%")
        # Creation of TIFF file
        with rasterio.open(output_file, 'w', **metadata) as dst:
            dst.write(array_fv, 1)  # Writing in band 1
        
        # LAI (reshape and write)
        reshaper = np.array(array_lai)
        #reshaper [scl>=8] = None
        reshaper = np.delete(reshaper,0,0) # delete last row
        reshaper = np.delete(reshaper,0,1) # delete first column
        array_lai = reshaper
        # Writing Output albedo
        output_file = "/home/eoafrica/shared/Final Workflow/LAI/Sidi_Bennour_LAI_{0}.tif".format(arr)
        #print("LAI: ",arr,"........100%")
        # Creation of TIFF file
        with rasterio.open(output_file, 'w', **metadata) as dst:
            dst.write(array_lai, 1)  # Writing in band 1
        
        print("+++ Date ",arr," is complete +++")
        
    else:
        print("too many nans in ",arr)

NDVI:  2021-10-08 ........100%
Albedo:  2021-10-08 ........100%
FVC:  2021-10-08 ........100%
LAI:  2021-10-08 ........100%
+++ Date  2021-10-08  is complete +++
NDVI:  2021-10-13 ........100%
Albedo:  2021-10-13 ........100%
FVC:  2021-10-13 ........100%
LAI:  2021-10-13 ........100%
+++ Date  2021-10-13  is complete +++
NDVI:  2021-10-18 ........100%
Albedo:  2021-10-18 ........100%
FVC:  2021-10-18 ........100%
LAI:  2021-10-18 ........100%
+++ Date  2021-10-18  is complete +++
NDVI:  2021-10-23 ........100%
Albedo:  2021-10-23 ........100%
FVC:  2021-10-23 ........100%
LAI:  2021-10-23 ........100%
+++ Date  2021-10-23  is complete +++
NDVI:  2021-10-28 ........100%
Albedo:  2021-10-28 ........100%
FVC:  2021-10-28 ........100%
LAI:  2021-10-28 ........100%
+++ Date  2021-10-28  is complete +++
NDVI:  2021-11-02 ........100%
Albedo:  2021-11-02 ........100%
FVC:  2021-11-02 ........100%
LAI:  2021-11-02 ........100%
+++ Date  2021-11-02  is complete +++
NDVI:  2021-11-07 ........10